# Table of Contents
1. [Context](#Context)
2. [Step by Step Guide](#Step-by-Step-Guide)

    2.1. [Obtaining and loading data](#Step-1-Obtaining-and-loading-data)

    2.2. [ERA5 and ERA5-Land processing](#step-2-era5-and-era5-land-data-processing)

    2.3. [EMO-1 data processing](#step-3-emo-1-data-processing)

    2.4. [Data merging](#step-4-data-merging)

    2.5. [Processing of combined ERA5-Land for BASD](#step-5-processing-of-combined-era5-land-for-bias-adjustment)

    2.6. [Processing of combined EMO-1](#step-6-processing-of-combined-emo-1-data)

    2.7. [Processing of combined EMO-1 for BASD](#step-7-processing-of-combined-emo-1-data-for-bias-adjustment)

    2.8. [Bias-adjustment and statistical downscaling](#step-8-bias-adjustment-and-statistical-downscaling)

    2.9. [Convert Bias-adjusted and downscaled ERA5 to EMO-1 format](#step-9-convert-bias-adjusted)

    2.10. [Final post-processing of ERA-5 files for consistency with EMO-1 data](#step-10-final-post-processing)
    
3. [Possible fixes to common issues](#possible-fixes)




# Context
This notebook will provide a step-by-step guide to perform bias adjustment and statistical downscaling with ISIMIP3BASD, ERA-5 Land, and EMO-1. Each step accompanies the code, explanation along with the necessary libraries required to run the code. To run the code, it is recommended to have Python version **3.11.6** installed along with the **Jupyter** Notebook extension.


# Step by Step Guide
<a id="Step-by-Step-Guide"></a>

### Workflow for merged all data

<div style="width:100%; display:block;">
    <div style="width:50%;"><img src="https://naturalhazards.eu/workflow.jpg"></div>
</div>

### Workflow for yearly data

<div style="width:100%; display:block;">
    <div style="width:50%;"><img src="https://naturalhazards.eu/workflow2.jpg"></div>
</div>

# Installation

### Environment setup

The CLIMB workflow uses two separate environments:

- a **core environment** for Steps 1–10 (`compass-env`),  
- a **LISVAP environment** for Step 11 (`lisvap-env`).

Both environments are defined in the repository and can be created with Conda.  
For users who prefer `pip` and virtual environments, a `requirements.txt` file is also provided.

---

### Core environment for Steps 1–10 (`compass-env`)

The main environment used for downloading, processing and bias-adjusting climate data  
(Steps 1–10) is defined in `environment.yml`.

This environment includes, among others:

- core scientific stack: `numpy`, `pandas`, `xarray`, `scipy`, `statsmodels`, `scikit-learn`, `dask`,  
- NetCDF/GRIB stack: `netcdf4`, `h5netcdf`, `h5py`, `eccodes`, `cfgrib`, `cftime`, `cf-units`, `iris`,  
- geospatial libraries: `gdal`, `geopandas`, `rasterio`, `rioxarray`, `pyogrio`, `shapely`, `pyproj`, `cartopy`,  
- visualisation and apps: `matplotlib`, `seaborn`, `plotly`, `folium`, `streamlit`, `jupyterlab`, `ipywidgets`,  
- climate data tools: `cdsapi`, `cads-api-client`, `climate_indices`, and `cdo` (Python wrapper).

#### Create and activate the environment

```bash
# From the environment folder of the repository
conda env create -f environment.yml
conda activate compass-env

After activation, you should be able to:
-	run the Jupyter notebooks (including the step-by-step guide),
-	execute the bash scripts for Steps 1–10,
-	use cdo, nco, cfgrib, eccodes and the geospatial stack directly from this environment.

If you need to update the environment after changes to environment.yml, use:

```bash
conda env update -f environment.yml --prune

---
### LISVAP environment for Step 11 (lisvap-env)

Step 11 (PET calculation) relies on the external LISVAP tool and PCRaster.<br/>
To keep the core workflow lightweight and to avoid conflicts, we recommend a separate environment for LISVAP, defined in environment-lisvap.yml.

This environment includes:
-	python=3.10,
-	pcraster 4.4.*,
-	numpy, pandas, xarray, netcdf4, eccodes, cfgrib, cftime,
-	and via pip: lisflood-lisvap==1.3.0, cdsapi, cads-api-client, climate_indices.

Create and activate the LISVAP environment
```bash
# From the environment folder of the repository
conda env create -f environment-lisvap.yml
conda activate lisvap-env


Once activated, you can follow the instructions in step_11/readme.txt to:
-	adapt the config.xml file,
-	point LISVAP to the output of Step 10 and to the basemap/ directory,
-	run LISVAP to generate PET products.

**Note: You do not need lisvap-env to run Steps 1–10.<br/>**
This environment is only required if you want to reproduce Step 11 (LISVAP-based PET).


---

### Alternative installation with pip (optional)

For users who prefer not to use Conda, we also provide a requirements.txt file that approximates the compass-env environment.

Important: Some packages (e.g. gdal, eccodes, cfgrib, pcraster) are much easier to install and manage via Conda. The pip-based installation is therefore recommended only for experienced users who can resolve system-level dependencies on their own.

Example: create a virtual environment and install with pip

```bash 
# Create and activate a virtual environment (Linux/macOS)
python -m venv climb-env
source climb-env/bin/activate

# or on Windows (PowerShell)
python -m venv climb-env
climb-env\Scripts\Activate.ps1

# Upgrade pip and install dependencies
pip install --upgrade pip
pip install -r requirements.txt

This should provide a working environment for running the notebooks and most parts of the workflow.<br/>
However, for LISVAP (Step 11) and for more complex geospatial operations, we still recommend using the Conda-based environments described above.

---

## Step 1. Obtaining and loading data
<a id="Step-1-Obtaining-and-loading-data"></a>

First, we need to download ERA-5 Land data from Copernicus Marine API. To access ERA-5 Land data via the Copernicus Marine API, you'll need to access it through the Copernicus Climate Data Store (CDS) API.

1. Go to the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/#!/home).

2. Create an account by registering if you haven't done so already.

3. Install the cdsapi

4. After registering, you'll receive an API key. This key needs to be saved in a .cdsapirc file in your home directory (~/.cdsapirc). More details on [https://cds.climate.copernicus.eu/how-to-api]

5. Replace your_username and your_api_key with your actual username and API key from your CDS account.

6. Accept the "Terms of use" on the end of the manage licenses page: [https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=download#manage-licences]

The cdsapi is a Python package that allows you to download data programmatically.

Install the cdsapi by running the following command

In [ ]:
pip install cdsapi

The **.cdsapirc** file should look like this:

In [ ]:
url: https://cds.climate.copernicus.eu/api/v2
key: your_username:your_api_key

You can now use the CDS API to download the ERA-5 Land data using the below code. You need to replace the relative path in the code with the path of your local directory where you would like to save the downloaded files.

In [ ]:
output_dir = 'YOUR-LOCATION/CLIMB'  # Keep location of Python to CLIMB folder

In [ ]:
import cdsapi
import os

c = cdsapi.Client()

years = range(1950, 2022)
months = range(1, 12)

for y in years:
    for m in months:
        era5land_file = os.path.join(output_dir, 'step_1', 'ERA5Land_' + str(y) + '_' + str(m) + '.grib')
        if os.path.isfile(era5land_file):
            print('Already downloaded:', era5land_file)
        else:
            c.retrieve(
                'reanalysis-era5-land',
                {
                    'variable': [
                        '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                        '2m_temperature', 'surface_solar_radiation_downwards', 'total_precipitation'
                    ],
                    'year': str(y),
                    'month': str(m),
                    'day': [
                        '01', '02', '03', '04', '05', '06', '07', '08', '09',
                        '10', '11', '12', '13', '14', '15', '16', '17', '18',
                        '19', '20', '21', '22', '23', '24', '25', '26', '27',
                        '28', '29', '30', '31',
                    ],
                    'time': [
                        '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00', '21:00', '22:00', '23:00',
                    ],
                    'area': [
                        55, 13, 48, 26,  # Poland and catchments of Polish rivers
                    ],
                    'format': 'grib',
                },
                era5land_file)

        era5_file = os.path.join(output_dir, 'step_1', 'ERA5_' + str(y) + '_' + str(m) + '.grib')
        if os.path.isfile(era5_file):
            print('Already downloaded:', era5_file)
        else:
            c.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': [
                        '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                        '2m_temperature', 'surface_solar_radiation_downwards', 'total_precipitation'
                    ],
                    'year': str(y),
                    'month': str(m),
                    'day': [    
                        '01', '02', '03', '04', '05', '06', '07', '08', '09',
                        '10', '11', '12', '13', '14', '15', '16', '17', '18',
                        '19', '20', '21', '22', '23', '24', '25', '26', '27',
                        '28', '29', '30', '31',
                    ],  
                    'time': [   
                        '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00', '21:00', '22:00', '23:00',
                    ],  
                    'area': [   
                        55, 13, 48, 26,
                    ],  
                    'format': 'grib',
                },
                era5_file)

**Table: Description of the ERA5 and ERA5-Land variables.**

| Short Name | Full Name                                   | Description                                                                                           |
|------------|---------------------------------------------|-------------------------------------------------------------------------------------------------------|
| `u10`      | 10m U Component of Wind                     | The eastward (u) component of the wind at 10 meters above ground level.                             |
| `v10`      | 10m V Component of Wind                     | The northward (v) component of the wind at 10 meters above ground level.                            |
| `dewpoint_temperature` | 2m Dewpoint Temperature              | The dew point temperature at 2 meters above ground level.                                            |
| `t2m`      | 2m Temperature                              | The air temperature at 2 meters above ground level.                                                  |
| `ssrd`    | Surface Solar Radiation Downwards          | The amount of solar radiation reaching the Earth's surface.                                          |
| `tp`       | Total Precipitation                        | The total amount of precipitation (rain, snow, etc.) over a given period.                           |


Once the data has finished downloading, it is now ready for processing.

## Step 2. ERA5 and ERA5-Land data processing
<a id="Step-2-ERA5processing"></a>

The script below will automate the necessary processing of downloaded ERA5-Land climate data for each year from 1950 to 2023. It converts monthly GRIB files into daily NetCDF files, calculates various climate variables (temperature, precipitation, solar radiation, wind speed, relative humidity), and performs necessary adjustments and cleanup. 

**NOTE:**: You must have the following libraries installed:
1. `cdo` either by sudo apt-get install cdo or follow the official documentation.
2. Make sure the bash is updated by use of sudo apt update if you are using WSL2.

In [ ]:
%%bash

for y in {1950..2023}
do
    year="$y"
    year_n="$((y+1))"
    i1="${output_dir}/step_1/ERA5Land_${year}_1.grib"
    i2="${output_dir}/step_1/ERA5Land_${year}_2.grib"
    i3="${output_dir}/step_1/ERA5Land_${year}_3.grib"
    i4="${output_dir}/step_1/ERA5Land_${year}_4.grib"
    i5="${output_dir}/step_1/ERA5Land_${year}_5.grib"
    i6="${output_dir}/step_1/ERA5Land_${year}_6.grib"
    i7="${output_dir}/step_1/ERA5Land_${year}_7.grib"
    i8="${output_dir}/step_1/ERA5Land_${year}_8.grib"
    i9="${output_dir}/step_1/ERA5Land_${year}_9.grib"
    i10="${output_dir}/step_1/ERA5Land_${year}_10.grib"
    i11="${output_dir}/step_1/ERA5Land_${year}_11.grib"
    i12="${output_dir}/step_1/ERA5Land_${year}_12.grib"
    i13="${output_dir}/step_1/ERA5Land_${year_n}_1.grib"
    o_tas="${output_dir}/step_2/ERA5_land_daily/tas_ERA5_${year}.nc"
    o_tasmin="${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_${year}.nc"
    o_tasmax="${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_${year}.nc"
    o_t_dew="${output_dir}/step_2/ERA5_land_daily/dew_ERA5_${year}.nc"
    o_sfcWind="${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_${year}.nc"
    o_rsds="${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_${year}.nc"
    o_pr="${output_dir}/step_2/ERA5_land_daily/pr_ERA5_${year}.nc"
    o_rsds_t="${output_dir}/step_2/ERA5_land_daily/rsds_t_ERA5_${year}.nc"
    o_pr_t="${output_dir}/step_2/ERA5_land_daily/pr_t_ERA5_${year}.nc"
    o_hurs="${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_${year}.nc"

    cdo -f nc -daymean -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tas
    cdo -f nc -daymin -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tasmin
    cdo -f nc -daymax -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tasmax
    cdo -f nc expr,rsds="var169/86400" -selhour,0 -selname,var169 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $i13 $o_rsds_t
    cdo -f nc expr,pr="var228/86.4" -selhour,0 -selname,var228 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $i13 $o_pr_t
    cdo selyear,$y -shifttime,-1days $o_rsds_t $o_rsds
    cdo selyear,$y -shifttime,-1days $o_pr_t $o_pr
    cdo -f nc -daymean -selname,var168,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_t_dew
    cdo expr,hurs="(10 ^ (7.5 * (var168-273.15) / (237.3+(var168-273.15)))) / (10 ^ (7.5 * (var167-273.15) / (237.3+(var167-273.15)))) * 100" $o_t_dew $o_hurs
    cdo -f nc expr,sfcWind="sqrt(var165*var165+var166*var166)" -daymean -selname,var165,var166 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_sfcWind
    rm $o_rsds_t
    rm $o_pr_t
    rm $o_t_dew

done


Now, in the similar way, run the script below will automate the necessary processing of downloaded ERA5 climate data for each year from 1990 to 2022. It converts monthly GRIB files into daily NetCDF files, calculates various climate variables (temperature, precipitation, solar radiation, wind speed, relative humidity), and performs necessary adjustments and cleanup. Similarly, replace {your path} with your local directory path.

This data will be helpful to gap-fill the ERA5-Land data in later stages of the workflow.

In [ ]:
%%bash

for y in {1950..2023}
do
    year="$y"
    i1="${output_dir}/step_1/ERA5_${year}_1.grib"
    i2="${output_dir}/step_1/ERA5_${year}_2.grib"
    i3="${output_dir}/step_1/ERA5_${year}_3.grib"
    i4="${output_dir}/step_1/ERA5_${year}_4.grib"
    i5="${output_dir}/step_1/ERA5_${year}_5.grib"
    i6="${output_dir}/step_1/ERA5_${year}_6.grib"
    i7="${output_dir}/step_1/ERA5_${year}_7.grib"
    i8="${output_dir}/step_1/ERA5_${year}_8.grib"
    i9="${output_dir}/step_1/ERA5_${year}_9.grib"
    i10="${output_dir}/step_1/ERA5_${year}_10.grib"
    i11="${output_dir}/step_1/ERA5_${year}_11.grib"
    i12="${output_dir}/step_1/ERA5_${year}_12.grib"

    o_t_dew="${output_dir}/step_2/ERA5_for_gapfill/dew_ERA5_${year}.nc"
    o_tas="${output_dir}/step_2/ERA5_for_gapfill/tas_ERA5_${year}.nc"
    o_tasmin="${output_dir}/step_2/ERA5_for_gapfill/tasmin_ERA5_${year}.nc"
    o_tasmax="${output_dir}/step_2/ERA5_for_gapfill/tasmax_ERA5_${year}.nc"
    o_sfcWind="${output_dir}/step_2/ERA5_for_gapfill/sfcWind_ERA5_${year}.nc"
    o_rsds="${output_dir}/step_2/ERA5_for_gapfill/rsds_ERA5_${year}.nc"
    o_pr="${output_dir}/step_2/ERA5_for_gapfill/pr_ERA5_${year}.nc"
    o_hurs="${output_dir}/step_2/ERA5_for_gapfill/hurs_ERA5_${year}.nc"

	# temperature
    cdo -f nc expr,dpt="var168" -daymean -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tas
    cdo -f nc expr,tx="var167" -daymin -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tasmin
    cdo -f nc expr,tn="var167" -daymax -selname,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_tasmax
	# accumulations in ERA5 are hourly, different from ERA5-land    
    cdo -f nc expr,rg="var169*24/86400" -daymean -selname,var169 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_rsds
    cdo -f nc expr,pr="var228*24/86.4" -daymean -selname,var228 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_pr
	# convert to relative humidity    
    cdo -f nc -daymean -selname,var168,var167 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_t_dew
    cdo expr,hurs="(10 ^ (7.5 * (var168-273.15) / (237.3+(var168-273.15)))) / (10 ^ (7.5 * (var167-273.15) / (237.3+(var167-273.15)))) * 100" $o_t_dew $o_hurs
    cdo -f nc expr,ws="sqrt(var165*var165+var166*var166)" -daymean -selname,var165,var166 -mergetime $i1 $i2 $i3 $i4 $i5 $i6 $i7 $i8 $i9 $i10 $i11 $i12 $o_sfcWind
	# remove temp files
    rm $o_t_dew
done


## Step 3. EMO-1 data processing
<a id="Step-3-EMO1processing"></a>

The European Meteorological Observations (EMO) dataset is a high-resolution, gridded meteorological dataset for Europe, offering both sub-daily and daily data across multiple variables. Based on historical and real-time observations, EMO is a product of the Copernicus Emergency Management Service. The dataset includes daily totals for precipitation, minimum and maximum temperatures, wind speed, solar radiation, and water vapor pressure. Additionally, EMO provides 6-hourly data for precipitation and mean temperature. EMO-1 offers grids with a spatial resolution of 1arcminx1arcmin (approximately 1.5 km), covering the period from 1990 to 2022.

To use the EMO-1 dataset for bias adjustment and downscaling with ISIMIP3BASD, it needs to be processed in the similar way as ERA5 data. First, the variables need to be processed using the script below.

### Download EMO-1 Data
Script use this url https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/CEMS-EFAS/meteorological_forcings/EMO-1arcmin/ for downloading and saving all files in separate folders in step 3

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# Retrieve the list of NetCDF files available in a given EMO-1 subdirectory
def get_file_list(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all links leading to .nc files
    file_links = [link.get('href') for link in soup.find_all('a') if link.get('href').endswith('.nc')]
    return file_links

# Download a single NetCDF file from EMO-1 and save it locally
def download_file(file_url, save_directory):
    response = requests.get(file_url)
    filename = os.path.join(save_directory, file_url.split('/')[-1])
    
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded: {file_url}")

# Iterate over the requested EMO-1 variable folders and download all files
def download_files_from_specific_folders(base_url, save_directory, folders):
    for folder in folders:
        full_folder_url = f"{base_url}/{folder}"
        print(f"Processing folder: {full_folder_url}")
        
        # Create the subdirectory if it does not exist
        folder_save_directory = os.path.join(save_directory, folder)
        if not os.path.exists(folder_save_directory):
            os.makedirs(folder_save_directory)
        
        file_list = get_file_list(full_folder_url)
        
        for file_link in file_list:
            full_file_url = f"{full_folder_url}/{file_link}"
            download_file(full_file_url, folder_save_directory)

# Usage
base_url = 'https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/CEMS-EFAS/meteorological_forcings/EMO-1arcmin'
save_directory = os.path.join(output_dir, "step_3", "emo_data")
# EMO-1 variable folders to download (can be adapted by the user)
folders = ['pr', 'rg', 'tn', 'tx', 'ws']  # List of folders to iterate through

# Create the main EMO-1 download directory if it does not exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

download_files_from_specific_folders(base_url, save_directory, folders)


### Cut EMO-1 to same dimensions as ERA5

In [ ]:
import os
import numpy as np
from netCDF4 import Dataset

#
# The script subsets the original EMO-1 files downloaded from
# https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/
# to a fixed latitude/longitude window (here: Poland). It iterates over all
# variable subfolders and writes the cropped files to a separate directory
# to avoid permission issues when overwriting .nc files in-place.
#

input_dir = os.path.join(output_dir, "step_3", "emo_data")
output_subdir = os.path.join(output_dir, "step_3", "emo_data", "cutted_emo")

# Geographic bounds for the Poland domain (can be adjusted by the user)
lon_min, lon_max = 12.950000, 26.050000  #For Poland
lat_min, lat_max = 47.950000, 55.050000  #For Poland

# Size of the chunk along the first non-lat/lon dimension (e.g. time)
chunk_size = 100

# Copy a variable in chunks, subsetting along lat/lon indices to reduce memory usage
def process_variable_in_chunks(src_var, dst_var, lon_indices, lat_indices):
    full_shape = src_var.shape
    dim_names = src_var.dimensions
    lat_dim = dim_names.index('lat') if 'lat' in dim_names else None
    lon_dim = dim_names.index('lon') if 'lon' in dim_names else None
    src_slices = [slice(None)] * len(full_shape)
    dst_slices = [slice(None)] * len(full_shape)
    
    if lat_dim is not None:
        src_slices[lat_dim] = lat_indices
        dst_slices[lat_dim] = slice(None)
    if lon_dim is not None:
        src_slices[lon_dim] = lon_indices
        dst_slices[lon_dim] = slice(None)
    if lat_dim is None and lon_dim is None:
        dst_var[:] = src_var[:]
        return

    chunk_dims = [i for i, dim in enumerate(dim_names) if dim not in ['lat', 'lon']]
    chunk_dim = chunk_dims[0] if chunk_dims else 0
    
    for start in range(0, full_shape[chunk_dim], chunk_size):
        end = min(start + chunk_size, full_shape[chunk_dim])
        src_chunk_slices = list(src_slices)
        dst_chunk_slices = list(dst_slices)
        src_chunk_slices[chunk_dim] = slice(start, end)
        dst_chunk_slices[chunk_dim] = slice(start, end)
        
        chunk_data = src_var[tuple(src_chunk_slices)]
        dst_var[tuple(dst_chunk_slices)] = chunk_data

# Open an EMO-1 file, subset it to the Poland domain, and write the result
def cut_file_for_poland(input_file_path, output_file_dir):
    try:
        with Dataset(input_file_path, 'r') as src:
            lon = src.variables['lon'][:]
            lat = src.variables['lat'][:]
            lon_indices = np.where((lon >= lon_min) & (lon <= lon_max))[0]
            lat_indices = np.where((lat >= lat_min) & (lat <= lat_max))[0]
            
            output_file_name = os.path.basename(input_file_path).replace('.nc', '_poland.nc')
            output_file_path = os.path.join(output_file_dir, output_file_name)
            
            with Dataset(output_file_path, 'w') as dst:
                dst.setncatts({a: src.getncattr(a) for a in src.ncattrs()})
                for name, dimension in src.dimensions.items():
                    if name == 'lon':
                        dst.createDimension(name, len(lon_indices))
                    elif name == 'lat':
                        dst.createDimension(name, len(lat_indices))
                    else:
                        dst.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))
                
                for name, variable in src.variables.items():
                    if name in ['lon', 'lat']:
                        x = dst.createVariable(name, variable.datatype, (name,))
                    else:
                        x = dst.createVariable(name, variable.datatype, variable.dimensions)
                    
                    dst[name].setncatts({a: variable.getncattr(a) for a in variable.ncattrs()})
                    
                    if name == 'lon':
                        dst[name][:] = lon[lon_indices]
                    elif name == 'lat':
                        dst[name][:] = lat[lat_indices]
                    else:
                        process_variable_in_chunks(src[name], dst[name], lon_indices, lat_indices)

        print(f"Successfully created cut file: {output_file_path}")
    except Exception as e:
        print(f"Failed to process file {input_file_path}. Error: {e}")
        raise

# Process all NetCDF files in a given EMO-1 variable folder
def process_folder(folder_path):
    try:
        output_subfolder = os.path.join(output_subdir, f"Cutted_{os.path.basename(folder_path)}")
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)
        
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".nc"):
                file_path = os.path.join(folder_path, file_name)
                
                print(f"Processing file: {file_path}")
                cut_file_for_poland(file_path, output_subfolder)
    except Exception as e:
        print(f"Error processing folder {folder_path}. Error: {e}")

# Main driver: loop over all selected EMO-1 variable folders for Poland
def main():
    try:
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)
        
        subfolders = ['pd', 'pr', 'ws', 'rg', 'tn', 'tx']
        
        for subfolder in subfolders:
            folder_path = os.path.join(input_dir, subfolder)
            if os.path.isdir(folder_path):
                print(f"Processing folder: {folder_path}")
                process_folder(folder_path)
            else:
                print(f"Folder not found: {folder_path}")

    except Exception as e:
        print(f"Critical error in main processing loop. Error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
%%bash


# Loop over all years in the EMO-1–EFAS overlap period
for y in {1990..2022}
do
	year="$y"

    # File paths for daily maximum/minimum temperature (tx/tn) and outputs
	tx="${output_dir}/step_3/emo_data/cutted_emo/tx/EMO-1arcmin-tx_${year}.nc"
	tn="${output_dir}/step_3/emo_data/cutted_emo/tn/EMO-1arcmin-tn_${year}.nc"
	tasmax="${output_dir}/step_3/emo_data/EFAS_converted/tasmax_${year}.nc"
	tasmin="${output_dir}/step_3/emo_data/EFAS_converted/tasmin_${year}.nc"
    tas="${output_dir}/step_3/emo_data/EFAS_converted/tas_${year}.nc"

    # Convert EMO-1 tx/tn (°C) to tasmax/tasmin/tas (K)
	cdo -f nc4c -z zip expr,tasmax="tx + 273.15" $tx $tasmax
	cdo -f nc4c -z zip expr,tasmin="tn + 273.15" $tn $tasmin
	cdo -f nc4c -z zip expr,tas="(tx + tn) / 2 + 273.15" -merge $tx $tn $tas

    # Relative humidity: derive hurs from vapour pressure (pd) and mean temperature
	hurs0="${output_dir}/step_3/emo_data/EFAS_converted/hurs_raw_${year}.nc"
	hurs="${output_dir}/step_3/emo_data/EFAS_converted/hurs_${year}.nc"
	pd="${output_dir}/step_3/emo_data/cutted_emo/pd/pd_${year}.nc"

    # Compute RH (hurs) from vapour pressure (pd) and mean temperature, then cap at 100%
	cdo -f nc4c -z zip -expr,hurs="pd / (6.11 * 10 ^ (7.5 * ((tn+tx)/2) / (237.3+((tn+tx)/2) ) )) * 100" -merge $tn $tx -shifttime,-1days $pd $hurs0
	cdo -f nc4c -z zip -expr,hurs="(hurs > 100 ) ? 100 : hurs" $hurs0 $hurs

    # Near-surface wind speed: copy ws to sfcWind and align time to EMO-1 convention
	ws="${output_dir}/step_3/emo_data/cutted_emo/ws/ws_${year}.nc"
	sfcWind="${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_${year}.nc"

	cdo -f nc4c -z zip expr,sfcWind="ws" -shifttime,-1days $ws $sfcWind

    # Precipitation: convert from daily totals to daily mean rate (kg m-2 s-1) and shift time
	pr="${output_dir}/step_3/emo_data/cutted_emo/pr/pr_${year}.nc"
	pr_c="${output_dir}/step_3/emo_data/EFAS_converted/pr_${year}.nc"

	cdo -f nc4c -z zip expr,pr="pr/86400" -shifttime,-1days $pr $pr_c

    # Shortwave radiation: convert from J m-2 to W m-2 and shift time
	rg="${output_dir}/step_3/emo_data/cutted_emo/rg/rg_${year}.nc"
	rsds="${output_dir}/step_3/emo_data/EFAS_converted/rsds_${year}.nc"

	cdo -f nc4c -z zip expr,rsds="rg/86400" -shifttime,-1days $rg $rsds

done

The scripts performs the following actions:

1. **Temperature Conversion:**
Converts maximum (`tx`) and minimum (`tn`) temperatures from Celsius to Kelvin by adding 273.15.
Calculates mean temperature (`tas`) as the average of `tx` and `tn`, then converts to Kelvin.

2. **Relative Humidity Calculation:**
Computes relative humidity (`hurs`) using temperature and partial pressure data.
The calculation uses the Magnus formula for saturation vapor pressure.
Limits the relative humidity to a maximum of 100%.

3. **Wind Speed Conversion:**
Converts wind speed (`ws`) to surface wind (`sfcWind`) without changing values.

4. **Precipitation Rate Conversion:**
Converts precipitation (`pr`) from mm/day to mm/second by dividing by 86400 (seconds in a day).

5. **Solar Radiation Conversion:**
Converts global radiation (`rg`) to downward short-wave radiation flux (`rsds`) by dividing by 86400.

6. **Time Adjustment:**
Applies a one-day backward time shift to certain variables (`hurs`, `sfcWind`, `pr`, `rsds`).

## Step 4. Data merging
<a id="Step-4-merging"></a>

Now, the data has been pre-processed for ERA5-Land and EMO-1. Subsequently, the data will now be merged as merging of these files allows for easier handling of long-term climate data series, which is crucial for ease in running simulations in later stages of the process.

For this purpose, the script below is designed to consolidate climate data from different sources and time periods. It creates three merged files: two for ERA5-Land data (covering 1950-1989 and 1990-2022) and one for EMO-1 data (covering 1990-2022).

In [ ]:
%%bash

# Define the base directory for the output as a variable
STEP_4_OUTPUT="${output_dir}/step_4"
MERGE_ERA5="${STEP_4_OUTPUT}/merge_era5"
MERGE_EMO1="${STEP_4_OUTPUT}/merge_emo1"

# Create the output directories if they don't exist
mkdir -p $MERGE_ERA5
mkdir -p $MERGE_EMO1

## Merge files per variable for ERA5_Land; convert the script to cover all vars: tas, tasmin, tasmax, sfcWind, hurs, rsds, pr

## Merge ERA5_Land for the period which is available in EMO-1, i.e. 1990-2022
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_2022.nc $MERGE_ERA5/hurs_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_2022.nc $MERGE_ERA5/tas_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_2022.nc $MERGE_ERA5/tasmin_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_2022.nc $MERGE_ERA5/tasmax_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_2022.nc $MERGE_ERA5/sfcWind_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_2022.nc $MERGE_ERA5/rsds_ERA5_1990_2022_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_199?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_200?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_201?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_2020.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_2021.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_2022.nc $MERGE_ERA5/pr_ERA5_1990_2022_t.nc

## Merge ERA5_Land for the preceding period which is NOT available in EMO-1, i.e. 1950-1989
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_198?.nc $MERGE_ERA5/hurs_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/tas_ERA5_198?.nc $MERGE_ERA5/tas_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/tasmin_ERA5_198?.nc $MERGE_ERA5/tasmin_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/tasmax_ERA5_198?.nc $MERGE_ERA5/tasmax_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/sfcWind_ERA5_198?.nc $MERGE_ERA5/sfcWind_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/rsds_ERA5_198?.nc $MERGE_ERA5/rsds_ERA5_1950_1989_t.nc
cdo mergetime ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_195?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_196?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_197?.nc ${output_dir}/step_2/ERA5_land_daily/pr_ERA5_198?.nc $MERGE_ERA5/pr_ERA5_1950_1989_t.nc

## Merge EMO-1 files using relative paths
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/hurs_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/hurs_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/hurs_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/hurs_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/hurs_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/hurs_2022.nc $MERGE_EMO1/hurs_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/tas_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tas_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tas_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tas_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/tas_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/tas_2022.nc $MERGE_EMO1/tas_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmin_2022.nc $MERGE_EMO1/tasmin_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/tasmax_2022.nc $MERGE_EMO1/tasmax_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/sfcWind_2022.nc $MERGE_EMO1/sfcWind_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/rsds_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/rsds_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/rsds_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/rsds_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/rsds_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/rsds_2022.nc $MERGE_EMO1/rsds_1990_2022_t.nc
cdo -f nc4c -z zip mergetime ${output_dir}/step_3/emo_data/EFAS_converted/pr_199?.nc ${output_dir}/step_3/emo_data/EFAS_converted/pr_200?.nc ${output_dir}/step_3/emo_data/EFAS_converted/pr_201?.nc ${output_dir}/step_3/emo_data/EFAS_converted/pr_2020.nc ${output_dir}/step_3/emo_data/EFAS_converted/pr_2021.nc ${output_dir}/step_3/emo_data/EFAS_converted/pr_2022.nc $MERGE_EMO1/pr_1990_2022_t.nc


1. **ERA5-Land Data Merging (1990-2022)**:

Merges ERA5-Land relative humidity data files for the years 1990 to 2022.
Uses wildcard patterns (e.g., '199?.nc') to include all files for each decade.
The merged output is saved as, e.g: 'hurs_ERA5_1990_2022_t.nc'.

2. **ERA5-Land Data Merging (1950-1989)**:

Merges ERA5-Land relative humidity data files for the years 1950 to 1989.
This covers the period not available in EMO-1 dataset.
The merged output is saved as, e.g: 'hurs_ERA5_1950_1989_t.nc'.

3. **EMO-1 Data Merging (1990-2022)**:

Merges EMO-1 relative humidity data files for the years 1990 to 2022.
Uses the CDO (Climate Data Operators) tool with specific options:

'-f nc4c': Specifies NetCDF4 classic format output.
'-z zip': Applies zip compression to the output file.
The merged output is saved as, e.g: 'hurs_1990_2022_t.nc'.

## Step 5. Processing of combined ERA5-Land for bias-adjustment
<a id="Step-5-secondary-processing"></a>

The combined ERA5-Land data from step 4 for the year (1950-1989) and (1990-2022) will go through set of procedures to ensure that the dimensions of each file are in-accordance with the requirements of ISIMIP3BASD scripts. For this purpose, the below script will reorder the dimensions of the NetCDF file to lon,lat,time from time,lat,lon.


In [ ]:
%%bash

# Ensure the output directory for Step 5 exists
mkdir -p "${output_dir}/step_5"

n_lats=10
n_lons=10

# Number of chunks along latitude and longitude for ncpdq (can be tuned)
n_lats=10
n_lons=10

# Process each variable separately to prepare ERA5-Land input for BASD
# ----------------------------------------------------------------------
# hurs: compute number of time steps, then re-chunk and reorder dimensions
# ----------------------------------------------------------------------
#hurs
filename="${output_dir}/step_4/merge_era5/hurs_ERA5_1990_2022_t.nc"  #for merged data
#filename="${output_dir}/step_2/ERA5_land_daily/hurs_ERA5_2023.nc"  #for selected year
n_times=$(cdo ntime $filename | awk 'NR==1 {print $1}')
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,"$n_times" --cnk_dmn=lat,"$n_lats" --cnk_dmn=lon,"$n_lons" -a lon,lat,time "$filename" "${output_dir}/step_5/ERA5_$(basename "$filename" .nc).nc"

echo "n_times: $n_times"
echo "n_lats: $n_lats"
echo "n_lons: $n_lons"
echo "Filename: $filename"
echo "Output file: ${output_dir}/step_5/ERA5_$(basename "$filename" .nc)"

#
# ----------------------------------------------------------------------
# tas: re-chunk 2 m temperature (later renamed from 2t to tas with ncrename)
# ----------------------------------------------------------------------
#tas
##### After generate file remember to rename variable name to tas, same as EMO1
##### use this code in terminal: ncrename -v 2t,tas /mnt/g/compass/compass_framework/step_5/tas_YEAR.nc #replace input file with the tas file that you want to rename

filename2="${output_dir}/step_4/merge_era5/ERA5_land_daily/tas_ERA5_1990_2022_t.nc"
n_times=$(cdo ntime $filename2 | awk 'NR==1 {print $1}')
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename2 "${output_dir}/step_5/ERA5_$(basename "$filename2" .nc).nc"
#
echo "n_times: $n_times"
echo "n_lats: $n_lats"
echo "n_lons: $n_lons"
echo "Filename: $filename2"
echo "Output file: ${output_dir}/step_5/ERA5_$(basename "$filename2" .nc)"


#
# ----------------------------------------------------------------------
# sfcWind: re-chunk near-surface wind speed
# ----------------------------------------------------------------------
##sfcWind
filename3="${output_dir}/step_4/merge_era5/ERA5_land_daily/sfcWind_ERA5_1990_2022_t.nc"
n_times=$(cdo ntime $filename3 | awk 'NR==1 {print $1}')
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename3 "${output_dir}/step_5/ERA5_$(basename "$filename3" .nc).nc"
#
echo "n_times: $n_times"
echo "n_lats: $n_lats"
echo "n_lons: $n_lons"
echo "Filename: $filename3"
echo "Output file: ${output_dir}/step_5/ERA5_$(basename "$filename3" .nc)"


# ----------------------------------------------------------------------
# rsds: re-chunk shortwave radiation
# ----------------------------------------------------------------------
#rsds
filename4="${output_dir}/step_4/merge_era5/ERA5_land_daily/rsds_ERA5_1990_2022_t.nc"
n_times=$(cdo ntime $filename4 | awk 'NR==1 {print $1}')
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename4 "${output_dir}/step_5/ERA5_$(basename "$filename4" .nc).nc"
#
echo "n_times: $n_times"
echo "n_lats: $n_lats"
echo "n_lons: $n_lons"
echo "Filename: $filename4"
echo "Output file: ${output_dir}/step_5/ERA5_$(basename "$filename4" .nc)"

# ----------------------------------------------------------------------
# pr: re-chunk precipitation
# ----------------------------------------------------------------------
#pr
filename5="${output_dir}/step_4/merge_era5/ERA5_land_daily/pr_ERA5_1990_2022_t.nc"
n_times=$(cdo ntime "$filename5" | awk 'NR==1 {print $1}')
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename5 "${output_dir}/step_5/ERA5_$(basename "$filename5" .nc).nc"

echo "n_times: $n_times"
echo "n_lats: $n_lats"
echo "n_lons: $n_lons"
echo "Filename: $filename5"
echo "Output file: ${output_dir}/step_5/ERA5_$(basename "$filename5" .nc)"


#
# ----------------------------------------------------------------------
# Derived variables: tasrange and tasskew from tas, tasmin (tn) and tasmax (tx)
# ----------------------------------------------------------------------
#Convert tas to tasrange and tasskew
tas="${output_dir}/step_4/merge_era5/ERA5_land_daily/tas_ERA5_1990_2022_t.nc"
tn="${output_dir}/step_4/merge_era5/ERA5_land_daily/tasmin_ERA5_1990_2022_t.nc"
tx="${output_dir}/step_4/merge_era5/ERA5_land_daily/tasmax_ERA5_1990_2022_t.nc"
tasrange="${output_dir}/step_4/merge_era5/ERA5_land_daily/tasrange_ERA5_1990_2022_t.nc"
tasskew="${output_dir}/step_4/merge_era5/ERA5_land_daily/tasskew_ERA5_1990_2022_t.nc"
# Use the tas file to determine the number of time steps for tasrange/tasskew
n_times=$(cdo ntime "$tas" | awk 'NR==1 {print $1}')
##
cdo expr,tasrange="tx - tn" -merge -chname,2t,tn $tn -chname,2t,tx $tx $tasrange
cdo expr,tasskew=" ( tas - tn ) / ( tx - tn ) " -merge -chname,2t,tn $tn -chname,2t,tx $tx -chname,2t,tas $tas $tasskew
##
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $tasrange "${output_dir}/step_5/ERA5_$(basename "$tasrange" .nc).nc"
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $tasskew "${output_dir}/step_5/ERA5_$(basename "$tasskew" .nc).nc"
#

The script also generates new variables for the temperature calculations (`tasrange` & `tasskew`):

a. `tasrange`: Difference between maximum and minimum temperatures.

b. `tasskew`: Normalized temperature, indicating where the mean temperature falls between the daily minimum and maximum.

## Step 6. Processing of combined EMO-1 data
<a id="Step-6-secondary-processing-emo-1"></a>

The combined EMO-1 data from step 4 will be scaled to the same resolution as ERA5-Land to ensure consistency in bias_adjustment and downscaling procedure. The procedure involves generating a grid file with of ERA5Land and remapping the generated weight file from EMO-1 file based on the grid_file. Finally, an aggregated file is generated which is remapped based on the grid_file from ERA5-Land ensuring similar resolution.

In [ ]:
%%bash

# Ensure the output directory for Step 6 exists
mkdir -p "${output_dir}/step_6"

# Convert high-resolution EMO-1 fields to the ERA5-Land grid (same resolution)

# For each variable, derive the target grid description from the merged ERA5-Land file.
# The resulting grid file is reused for EMO-1 remapping for that variable and period.

# List of variables to be remapped (can be adapted by the user)
variables=("tas" "pr" "rsds" "sfcWind" "tasmax" "tasmin" "hurs")

## Loop to process each variable
for var in "${variables[@]}"; do
    # ERA5-Land file used to define the target grid
    era5_name="${output_dir}/step_4/merge_era5/${var}_ERA5_1990_2022_t.nc"  #file from step 4 or if it is only one year from step 2
    grid_file="${output_dir}/step_4/merge_era5/${var}_ERA5_1990_2022_t_aggregate.txt"
    cdo griddes $era5_name > $grid_file

    # Generate conservative remapping weights from EMO-1 native grid to the ERA5-Land grid
    emo1_file="${output_dir}/step_4/merge_emo1/${var}_1990_2022_t.nc"
    weight_file="${output_dir}/step_4/merge_emo1/remap_weight_${var}_1990_2022_t_aggregate.nc"
    efas_grid="${output_dir}/step_4/merge_emo1/efas_grid.txt"
    cdo gencon,$grid_file -setgrid,$efas_grid $emo1_file $weight_file

    # Apply the conservative remapping to EMO-1 and write the remapped file
    cdo remap,$grid_file,$weight_file $emo1_file "${output_dir}/step_6/${var}_1990_2022_t_aggregate.nc"
done


## Step 7. Processing of combined EMO-1 data for bias adjustment
<a id="Step-7-secondary-processing-emo-1-ba"></a>

Repeating the bias adjustment processing procedure as described in step 5 for EMO-1 data now. Running the script below will convert the dimensions of EMO-1 data to proper format ready for bias adjustment and downscaling.

### Use only for merged data (for example 1990-2022)

In [ ]:
%%bash

# Ensure the output directory for Step 7 exists
mkdir -p "${output_dir}/step_7"

# Number of chunks along latitude and longitude for ncpdq
n_lats=10
n_lons=10

# Variables to process
variables=("tas" "sfcWind" "hurs" "rsds" "pr")

# Two data types:
#  - ""          : non-aggregated daily files (..._1990_2022_t.nc, from Step 4)
#  - "aggregate" : aggregated daily files     (..._1990_2022_t_aggregate.nc, from Step 6)
types=("aggregate" "")

# ----------------------------------------------------------------------
# 1) Re-chunk core EMO-1 variables for both non-aggregated and aggregated data
# ----------------------------------------------------------------------
for type in "${types[@]}"; do
    for var in "${variables[@]}"; do
        if [ -n "$type" ]; then
            # Aggregated EMO-1 file from Step 6 (remapped to ERA5-Land grid)
            filename="${output_dir}/step_6/${var}_1990_2022_t_${type}.nc"
        else
            # Non-aggregated EMO-1 file from Step 4 (merged original EMO-1 grid)
            filename="${output_dir}/step_4/merge_emo1/${var}_1990_2022_t.nc"
        fi

        if [ -f "$filename" ]; then
            echo "Processing variable: $var, type: ${type:-non-aggregate}"
            # Determine the number of time steps for chunking
            n_times=$(cdo ntime "$filename" | awk 'NR==1 {print $1}')
            echo "  n_times: $n_times, n_lats: $n_lats, n_lons: $n_lons"
            echo "  Input file:  $filename"
            echo "  Output file: ${output_dir}/step_7/EFAS_$(basename "$filename" .nc).nc"

            # Re-chunk and reorder dimensions to lon,lat,time
            ncpdq -4 -O --cnk_plc=g3d \
                  --cnk_dmn=time,"$n_times" --cnk_dmn=lat,"$n_lats" --cnk_dmn=lon,"$n_lons" \
                  -a lon,lat,time "$filename" \
                  "${output_dir}/step_7/EFAS_$(basename "$filename" .nc).nc"
        else
            echo "File $filename does not exist, skipping..."
        fi
    done
done

# ----------------------------------------------------------------------
# 2) Derive tasrange and tasskew and re-chunk them for both data types
# ----------------------------------------------------------------------
for type in "${types[@]}"; do
    if [ -n "$type" ]; then
        # Aggregated EMO-1 files from Step 6
        tas="${output_dir}/step_6/tas_1990_2022_t_${type}.nc"
        tn="${output_dir}/step_6/tasmin_1990_2022_t_${type}.nc"
        tx="${output_dir}/step_6/tasmax_1990_2022_t_${type}.nc"
        tasrange="${output_dir}/step_6/tasrange_1990_2022_t_${type}.nc"
        tasskew="${output_dir}/step_6/tasskew_1990_2022_t_${type}.nc"
    else
        # Non-aggregated EMO-1 files from Step 4
        tas="${output_dir}/step_4/merge_emo1/tas_1990_2022_t.nc"
        tn="${output_dir}/step_4/merge_emo1/tasmin_1990_2022_t.nc"
        tx="${output_dir}/step_4/merge_emo1/tasmax_1990_2022_t.nc"
        tasrange="${output_dir}/step_4/merge_emo1/tasrange_1990_2022_t.nc"
        tasskew="${output_dir}/step_4/merge_emo1/tasskew_1990_2022_t.nc"
    fi

    if [ -f "$tn" ] && [ -f "$tx" ] && [ -f "$tas" ]; then
        echo "Deriving tasrange and tasskew for type: ${type:-non-aggregate}"

        # Calculate tasrange: absolute difference between tasmax and tasmin
        cdo -expr,tasrange="(( tasmax - tasmin ) > 0 ) ? ( tasmax - tasmin ) : ( tasmin - tasmax )" \
            -merge "$tn" "$tx" "$tasrange"

        # Calculate tasskew: relative position of tas between tasmin and tasmax
        cdo -expr,tasskew=" ( tas - tasmin ) / ( tasmax - tasmin ) " \
            -merge "$tn" "$tx" "$tas" "$tasskew"

        # Re-chunk tasrange and tasskew using their own time dimension
        for file in "$tasrange" "$tasskew"; do
            if [ -f "$file" ]; then
                # Determine the number of time steps for this derived file
                n_times=$(cdo ntime "$file" | awk 'NR==1 {print $1}')
                echo "  Re-chunking derived file: $file (n_times=$n_times)"

                ncpdq -4 -O --cnk_plc=g3d \
                      --cnk_dmn=time,"$n_times" --cnk_dmn=lat,"$n_lats" --cnk_dmn=lon,"$n_lons" \
                      -a lon,lat,time "$file" \
                      "${output_dir}/step_7/EFAS_$(basename "$file" .nc).nc"
            else
                echo "  Derived file $file does not exist, skipping..."
            fi
        done
    else
        echo "Files $tn, $tx or $tas do not exist for type ${type:-non-aggregate}, skipping..."
    fi
done


### For single-year data workflow (for example 1990 only)

In [ ]:
%%bash

# Number of netCDF chunks (do not change)
n_lats=10
n_lons=10

# List of variables to process
variables=("tas" "sfcWind" "hurs" "rsds" "pr")
types=("aggregate" "")  # 'aggregate' and the second type is empty

# Processing files for each variable and data type
for type in "${types[@]}"; do
    for var in "${variables[@]}"; do
        if [ -n "$type" ]; then
            filename="${output_dir}/step_6/${var}_1990_${type}.nc"
        else
            filename="${output_dir}/step_3/emo_data/EFAS_converted/${var}_1990.nc"
        fi
        
        if [ -f "$filename" ]; then  # Check if the file exists
            n_times=$(cdo ntime "$filename" | awk 'NR==1 {print $1}')
            ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename "${output_dir}/step_7/EFAS_$(basename "$filename" .nc).nc"
        else
            echo "File $filename does not exist, skipping..."
        fi
    done
done


# Processing tasmin and tasmax files to tasrange and tasskew for both data types
for type in "${types[@]}"; do
    if [ -n "$type" ]; then
        tas="${output_dir}/step_6/tas_1990_${type}.nc"
        tn="${output_dir}/step_6/tasmin_1990_${type}.nc"
        tx="${output_dir}/step_6/tasmax_1990_${type}.nc"
        tasrange="${output_dir}/step_6/tasrange_1990_${type}.nc"
        tasskew="${output_dir}/step_6/tasskew_1990_${type}.nc"
    else
        tas="${output_dir}/step_3/emo_data/EFAS_converted/tas_1990.nc"
        tn="${output_dir}/step_3/emo_data/EFAS_converted/tasmin_1990.nc"
        tx="${output_dir}/step_3/emo_data/EFAS_converted/tasmax_1990.nc"
        tasrange="${output_dir}/step_3/emo_data/EFAS_converted/tasrange_1990.nc"
        tasskew="${output_dir}/step_3/emo_data/EFAS_converted/tasskew_1990.nc"
    fi

    if [ -f "$tn" ] && [ -f "$tx" ]; then
        cdo -expr,tasrange="(( tasmax - tasmin ) > 0 ) ? ( tasmax - tasmin ) : ( tasmin - tasmax )" -merge $tn $tx $tasrange
        cdo -expr,tasskew=" ( tas - tasmin ) / ( tasmax - tasmin ) " -merge $tn $tx $tas $tasskew

        # Processing tasrange and tasskew files
        for file in "$tasrange" "$tasskew"; do
            n_times=$(cdo ntime "$filename" | awk 'NR==1 {print $1}')
            ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $file "${output_dir}/step_7/EFAS_$(basename "$file" .nc).nc"
        done
    else
        echo "Files $tn or $tx do not exist, skipping..."
    fi
done


## Step 8. Bias adjustment and statistical downscaling
<a id="Step-8-secondary-processing-emo-1-ba"></a>

The final step is to run the adapted script from Stefan Lange [ISIMIP3BASDv3.0.2](https://zenodo.org/records/7151476) for bias adjustment and statistical downscaling of ERA5-Land data based on finer resolution EMO-1 data that is processed in step 6 combined with the coarser resolution EMO-1 data which was converted to same resolution as ERA5-Land in step 6. By doing so, the ERA5-Land can be downscaled to same resolution as that of EMO-1. 

Note: The script is time-consuming and may take several days to complete depending on the computational resources. It is not recommended to stop the script during the process as it may corrupt the ouput files. Hence, it is recommended to run it on a hosted server or cloud based services if such resources are available.

In [ ]:
%%bash

## this is an adapted script from Stefan Lange for running BASD on ERA5Land and EMO1 files

# Variables and calibration period
vars="hurs pr rsds sfcWind tas tasrange tasskew"
per="1990_2022"

# Location of the ISIMIP3BASD code and input/output directories
cdir="${output_dir}/isimip3basd-master/code"
idir_era5="${output_dir}/step_5"
idir_emo="${output_dir}/step_7"
odir="${output_dir}/step_8"

# Create output directory if it does not exist
mkdir -p "$odir"

# Optional: activate a virtual/conda environment providing ISIMIP3BASD
# source /path/to/your/env/bin/activate

# Iterate over all variables
for var in $vars; do
  echo "==================================================================="
  echo "Processing variable: $var (period: ${per})"
  echo "==================================================================="

  # Define file paths
  # EMO-1 / EFAS:
  #   - obs_hist_fine   : high-resolution reference (non-aggregated)
  #   - obs_hist_coarse : aggregated reference on the coarse grid
  obs_hist_fine="${idir_emo}/EFAS_${var}_${per}_t.nc"
  obs_hist_coarse="${idir_emo}/EFAS_${var}_${per}_t_aggregate.nc"

  # ERA5-Land:
  #   - sim_hist_coarse : historical model series (overlap with EMO-1)
  #   - sim_fut_coarse  : here the same as sim_hist_coarse, since we
  #                       currently apply BASD to the 1990–2022 period only
  sim_hist_coarse="${idir_era5}/ERA5_${var}_ERA5_${per}_t.nc"
  sim_fut_coarse="${idir_era5}/ERA5_${var}_ERA5_${per}_t.nc"

  # Output files:
  sim_fut_basd_coarse="${odir}/ERA5_${var}_ERA5_${per}_t_ba.nc"
  sim_fut_basd_fine="${odir}/ERA5_${var}_ERA5_${per}_t_basd.nc"

  echo "  obs_hist_fine:   $obs_hist_fine"
  echo "  obs_hist_coarse: $obs_hist_coarse"
  echo "  sim_hist_coarse: $sim_hist_coarse"
  echo "  sim_fut_coarse:  $sim_fut_coarse"
  echo "  sim_fut_ba:      $sim_fut_basd_coarse"
  echo "  sim_fut_basd:    $sim_fut_basd_fine"
  echo

  # Set BASD options based on variable
  case $var in
    hurs*)
      options_ba="-v hurs --lower-bound 0 --lower-threshold .01 --upper-bound 100 --upper-threshold 99.99 -t bounded --unconditional-ccs-transfer 1 --trendless-bound-frequency 1"
      options_sd="-v hurs --lower-bound 0 --lower-threshold .01 --upper-bound 100 --upper-threshold 99.99"
      ;;
    pr*)
      options_ba="-v pr --lower-bound 0 --lower-threshold .0000011574 --distribution gamma -t mixed"
      options_sd="-v pr --lower-bound 0 --lower-threshold .0000011574"
      ;;
    rsds*)
      options_ba="-v rsds --lower-bound 0 --lower-threshold .0001 --upper-bound 1 --upper-threshold .9999 -t bounded -w 15"
      options_sd="-v rsds --lower-bound 0 --lower-threshold .01"
      ;;
    sfcWind*)
      options_ba="-v sfcWind --lower-bound 0 --lower-threshold .01 --distribution weibull -t mixed"
      options_sd="-v sfcWind --lower-bound 0 --lower-threshold .01"
      ;;
    tas)
      options_ba="-v tas --distribution normal -t additive -d 1"
      options_sd="-v tas"
      ;;
    tasrange)
      options_ba="-v tasrange --lower-bound 0 --lower-threshold .01 --distribution weibull -t mixed"
      options_sd="-v tasrange --lower-bound 0 --lower-threshold .01"
      ;;
    tasskew)
      options_ba="-v tasskew --lower-bound 0 --lower-threshold .0001 --upper-bound 1 --upper-threshold .9999 -t bounded"
      options_sd="-v tasskew --lower-bound 0 --lower-threshold .0001 --upper-bound 1 --upper-threshold .9999"
      ;;
    *)
      echo "Variable $var not supported ... aborting ..."
      exit 1
      ;;
  esac

  # -------------------------------------------------------------------
  # 1) Bias adjustment on the coarse grid (ERA5-Land resolution)
  # -------------------------------------------------------------------
  echo "Running bias_adjustment.py for $var ..."
  time python -u "${cdir}/bias_adjustment.py" $options_ba \
    --n-processes 16 \
    --randomization-seed 0 \
    --step-size 1 \
    -o "$obs_hist_coarse" \
    -s "$sim_hist_coarse" \
    -f "$sim_fut_coarse" \
    -b "$sim_fut_basd_coarse"

  chmod 664 "$sim_fut_basd_coarse"
  echo

  # -------------------------------------------------------------------
  # 2) Statistical downscaling to the EMO-1/EFAS grid
  # -------------------------------------------------------------------
  echo "Running statistical_downscaling.py for $var ..."
  time python -u "${cdir}/statistical_downscaling.py" $options_sd \
    --n-processes 16 \
    --randomization-seed 0 \
    -o "$obs_hist_fine" \
    -s "$sim_fut_basd_coarse" \
    -f "$sim_fut_basd_fine"

  chmod 664 "$sim_fut_basd_fine"
  echo
done

# Optional: deactivate environment
# deactivate


## Step 9. Convert Bias-adjusted and downscaled ERA5 to EMO-1 format
<a id="step-9-convert-bias-adjusted"></a>

### Step 9.1

The script will convert BASD ERA5-Land to EMO-1 format from 1950-1989 for every variable. If you are facing memory issues, refer to possible fixes section 1.4.

Use this script only for merged data (for example 1990-2022)

In [ ]:
%%bash

opath="${output_dir}/step_9"

## Convert BASD back to EMO-1 format; adapt the script to cover also the 1950-1989 period

# Wind
filename="${output_dir}/step_8/ERA5_sfcWind_ERA5_1950_1989_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {1950..1989}
do
  oname2="${opath}ws_${y}.nc"
  cdo -L -f nc4c -z zip expr,ws="sfcWind" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1 || { echo "Failed to remove $oname1"; exit 1; }

# Temperature
tas="${output_dir}/step_8/ERA5_tas_ERA5_1950_1989_t_basd.nc"
tasrange="${output_dir}/step_8/ERA5_tasrange_ERA5_1950_1989_t_basd.nc"
tas_a="${opath}t_$(basename "$tas" .nc).nc"
tasrange_a="${opath}t_$(basename "$tasrange" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $tas $tas_a
ncpdq -O --cnk_plc=uck -a time,lat,lon $tasrange $tasrange_a
for y in {1950..1989}
do
  tas_b="${opath}ta_$(basename "$tas" .nc)_${y}.nc"
  tasrange_b="${opath}ta_$(basename "$tasrange" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $tas_a $tas_b
  cdo -L -f nc4c -z zip -selyear,$y $tasrange_a $tasrange_b
  tx="${opath}tx_${y}.nc"
  tn="${opath}tn_${y}.nc"
  cdo -L -f nc4c -z zip expr,tx="tas + 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tx
  cdo -L -f nc4c -z zip expr,tn="tas - 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tn
  rm $tasrange_b # don't remove tas_b, needed later
done
rm $tas_a
rm $tasrange_a

# Radiation
filename="${output_dir}/step_8/ERA5_rsds_ERA5_1950_1989_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {1950..1989}
do
  oname2="${opath}rg_${y}.nc"
  cdo -L -f nc4c -z zip expr,rg="rsds * 86400" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1

# Vapour pressure / humidity
filename="${output_dir}/step_8/ERA5_hurs_ERA5_1950_1989_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {1950..1989}
do
  oname2="${opath}ta_$(basename "$filename" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $oname1 $oname2
  oname3="${opath}pd_${y}.nc"
  cdo -L -f nc4c -z zip expr,pd="hurs / 100 * (6.11 * 10 ^ (7.5 * (tas - 273.15) / (237.3 + (tas - 273.15))))" -merge -shifttime,1days -selyear,$y $oname2 -shifttime,630minutes $tas_b $oname3
done
rm $oname1

# Precipitation
filename="${output_dir}/step_8/ERA5_pr_ERA5_1950_1989_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {1950..1989}
do
  oname2="${opath}pr_${y}.nc"
  cdo -L -f nc4c -z zip mulc,86400 -shifttime,24hours -selyear,$y $oname1 $oname2
done
rm $oname1

### Step 9.2

The script will convert BASD ERA5-Land to EMO-1 format from 1990-2022.

Use this script only for merged data (for example 1990-2022)

In [ ]:
%%bash

opath="${output_dir}/step_9"

## convert BASD back to EMO-1 format; adapt the script to cover also the 1990-2022

# Wind
filename="${output_dir}/step_8/ERA5_sfcWind_ERA5_1990_2022_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {1990..2022}
do
  oname2="${opath}ws_${y}.nc"
  cdo -L -f nc4c -z zip expr,ws="sfcWind" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1 || { echo "Failed to remove $oname1"; exit 1; }

# Temperature
tas="${output_dir}/step_8/ERA5_tas_ERA5_1990_2022_t_basd.nc"
tasrange="${output_dir}/step_8/ERA5_tasrange_ERA5_1990_2022_t_basd.nc"
tas_a="${opath}t_$(basename "$tas" .nc).nc"
tasrange_a="${opath}t_$(basename "$tasrange" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $tas $tas_a
ncpdq -O --cnk_plc=uck -a time,lat,lon $tasrange $tasrange_a
for y in {1990..2022}
do
  tas_b="${opath}ta_$(basename "$tas" .nc)_${y}.nc"
  tasrange_b="${opath}ta_$(basename "$tasrange" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $tas_a $tas_b
  cdo -L -f nc4c -z zip -selyear,$y $tasrange_a $tasrange_b
  tx="${opath}tx_${y}.nc"
  tn="${opath}tn_${y}.nc"
  cdo -L -f nc4c -z zip expr,tx="tas + 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tx
  cdo -L -f nc4c -z zip expr,tn="tas - 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tn
  rm $tasrange_b # don't remove tas_b, needed later
done
rm $tas_a
rm $tasrange_a

## Radiation
filename="${output_dir}/step_8/ERA5_rsds_ERA5_1990_2022_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {1990..2022}
do
  oname2="${opath}rg_${y}.nc"
  cdo -L -f nc4c -z zip expr,rg="rsds * 86400" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1

## Vapour pressure / humidity
filename="${output_dir}/step_8/ERA5_hurs_ERA5_1990_2022_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {1990..2022}
do
  oname2="${opath}ta_$(basename "$filename" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $oname1 $oname2
  oname3="${opath}pd_${y}.nc"
  cdo -L -f nc4c -z zip expr,pd="hurs / 100 * (6.11 * 10 ^ (7.5 * (tas - 273.15) / (237.3+ (tas - 273.15) ) ))" -merge -shifttime,1days -selyear,$y $oname2 -shifttime,630minutes $tas_b $oname3
done
rm $oname1

## Precipitation
filename="${output_dir}/step_8/ERA5_pr_ERA5_1990_2022_t_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {1990..2022}
do
  oname2="${opath}pr_${y}.nc"
  cdo -L -f nc4c -z zip mulc,86400 -shifttime,24hours -selyear,$y $oname1 $oname2
done
rm $oname1

### Step 9.3 (optional for a single year, for example 2023)



In [ ]:
%%bash

opath="${output_dir}/step_9/"

###################################### 
#            1
######################################
 Temperature
tas="${output_dir}/step_8/ERA5_tas_ERA5_2023_basd.nc"
tasrange="${output_dir}/step_8/ERA5_tasrange_ERA5_2023_basd.nc"
tas_a="${opath}t_$(basename "$tas" .nc).nc"
tasrange_a="${opath}t_$(basename "$tasrange" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $tas $tas_a
ncpdq -O --cnk_plc=uck -a time,lat,lon $tasrange $tasrange_a
for y in {2023..2023}
do
  tas_b="${opath}ta_$(basename "$tas" .nc)_${y}.nc"
  tasrange_b="${opath}ta_$(basename "$tasrange" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $tas_a $tas_b
  cdo -L -f nc4c -z zip -selyear,$y $tasrange_a $tasrange_b
  tx="${opath}tx_${y}.nc"
  tn="${opath}tn_${y}.nc"
  cdo -L -f nc4c -z zip expr,tx="tas + 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tx
  cdo -L -f nc4c -z zip expr,tn="tas - 0.5 * tasrange - 273.15" -merge $tas_b $tasrange_b $tn
  rm $tasrange_b # don't remove tas_b, needed later
done
rm $tas_a
rm $tasrange_a


# Vapour pressure / humidity
filename="${output_dir}/step_8/ERA5_hurs_ERA5_2023_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {2023..2023}
do
  oname2="${opath}ta_$(basename "$filename" .nc)_${y}.nc"
  cdo -L -f nc4c -z zip -selyear,$y $oname1 $oname2
  oname3="${opath}pd_${y}.nc"
  cdo -L -f nc4c -z zip expr,pd="hurs / 100 * (6.11 * 10 ^ (7.5 * (tas - 273.15) / (237.3+ (tas - 273.15) ) ))" -merge -shifttime,1days -selyear,$y $oname2 -shifttime,630minutes $tas_b $oname3
done
rm $oname1
#



In [ ]:
%%bash

###################################### 
#            2
######################################

# Radiation
filename="${output_dir}/step_8/ERA5_rsds_ERA5_2023_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {2023..2023}
do
  oname2="${opath}rg_${y}.nc"
  cdo -L -f nc4c -z zip expr,rg="rsds * 86400" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1

# Wind
filename="${output_dir}/step_8/ERA5_sfcWind_ERA5_2023_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1

for y in {2023..2023}
do
  oname2="${opath}ws_${y}.nc"
  cdo -L -f nc4c -z zip expr,ws="sfcWind" -shifttime,1days -selyear,$y $oname1 $oname2
done
rm $oname1 || { echo "Failed to remove $oname1"; exit 1; }

#
# Precipitation
filename="${output_dir}/step_8/ERA5_pr_ERA5_2023_basd.nc"
oname1="${opath}t_$(basename "$filename" .nc).nc"
ncpdq -O --cnk_plc=uck -a time,lat,lon $filename $oname1
for y in {2023..2023}
do
  oname2="${opath}pr_${y}.nc"
  cdo -L -f nc4c -z zip mulc,86400 -shifttime,24hours -selyear,$y $oname1 $oname2
done
rm $oname1

## Step 10. Final post-processing of ERA-5 files for consistency with EMO-1 data
<a id="step-10-final-post-processing"></a>

<img src="https://naturalhazards.eu/timeshift.png">

### Step 10.1
The script defines input, output, and intermediate file paths for the processing of meteorological variables (1950 to 2022). The ipath, epath, and opath variables designate paths for the input files, ERA5 climate data, and output files, respectively.

The script performs several tasks in a loop over each year. It starts by computing a land mask for the BASD dataset, then extracts grid information and calculates regridding weights for the ERA5 data. For each year, the script processes the variable files by setting up file names for input and output. It handles the regridding of ERA5 files to match the BASD grid, adjusting for time shifts where necessary. It converts relative humidity (hurs) into partial pressure (pd) for certain variables and ensures consistency in units and time shifts across files.

The script also corrects the time vector, processes and compresses data using cdo commands for specific meteorological variables, and applies transformations such as packing values into smaller byte formats and adjusting chunking for NetCDF outputs. The time units are adapted depending on the time period (1950-1989 or 1990-2022), and temporary files are removed at the end of each iteration.

The script requires specific adjustments for each variable where the user needs to add some expressions and different timestamps to modify the data to match with that of EMO1. The code snippets for each variable has been provided seperately below for convenience.

The below script is for `ws` variable

In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}ws_1951.nc" # from step 9
basd_mask="${opath}basd_landmask.nc"
cdo -f nc4c -z zip setmisstoc,0 -expr,sfcWind="(ws >= 0) ? 1 : 0" -seltimestep,1 $basd_file $basd_mask
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}tasmax_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=990
n_lons=1510

## loop to repeat per variable
for y in {1950..2020}
do
	filename="${ipath}ws_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}sfcWind_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_sfcWind_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

	cdo -f nc4c -z zip remap,$grid_file,$weight_file -shifttime,1days $era5_file $era5_file_regrid


	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,-330minutes $oname1 $oname2

	ncatted -O -a _FillValue,ws,o,s,-9999 $oname2
	ncatted -O -a missing_value,ws,o,s,-9999 $oname2

	ncap2 -v -O -s 'ws=pack(ws,0.1,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-24)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1990-01-01 00:00:00" $oname5
	## 1950-1989: ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5

  # remove temporary files
	rm t*_ws_*.nc
done

To verify if the time steps are correct, use command cdo info {path_to_nc.nc} and do the same for EMO-1 file to confirm that the timestamps are consistent.

### Step 10.2
The initial code needs to be modified with expr command and other modifications necessary to add consistency in the units between EMO-1 and ERA-5 data. Therefore, below are the changes that need to be made to ensure that every variable is gap-filled and processed without any errors.

`pd`:





In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}pd_1951.nc" # from step 9
basd_mask="${opath}basd_landmask_pl.nc"
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}hurs_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=142
n_lons=262

## loop to repeat per variable
for y in {1950..1989}
do
	filename="${ipath}pd_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}hurs_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_hurs_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

  	era5_file_tas="${epath}tas_ERA5_${tile}.nc"
	era5_file_pd="${epath}pd_ERA5_${tile}.nc"

	"""
	Added expr,pd="hurs / 100 * (6.11 * 10 ^ (7.5 * (dpt - 273.15) / (237.3 + (dpt - 273.15) ) ))" 
	to calculate the actual vapor pressure which is used in EMO-1 data.

	"""
	cdo -f nc4c -z zip expr,pd="hurs / 100 * (6.11 * 10 ^ (7.5 * (dpt - 273.15) / (237.3 + (dpt - 273.15) ) ))" -merge $era5_file $era5_file_tas $era5_file_pd

	cdo -f nc4c -z zip remap,$grid_file,$weight_file -shifttime,1days $era5_file_pd $era5_file_regrid

	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,-690minutes $oname1 $oname2

	ncatted -O -a _FillValue,pd,o,s,-9999 $oname2
	ncatted -O -a missing_value,pd,o,s,-9999 $oname2

	ncap2 -v -O -s 'pd=pack(pd,0.1,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-24)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5
	#ncatted -a units,time,o,c,"days since 1990-01-01 00:00:00" $oname5
	
  # remove temporary files
rm t*_pd*.nc
done

`pr`

In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}pr_1951.nc" # from step 9
basd_mask="${opath}basd_landmask_pl.nc"
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}pr_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=142
n_lons=262

## loop to repeat per variable
for y in {1950..1989}
do
	filename="${ipath}pr_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}pr_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_pr_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

	"""
	Added -expr,pr="pr*86400" for consistency between the units in EMO-1 and ERA-5
	
	"""

	cdo -f nc4c -z zip remap,$grid_file,$weight_file -expr,pr="pr*86400" -shifttime,1days $era5_file $era5_file_regrid

	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,360minutes $oname1 $oname2

	ncatted -O -a _FillValue,pr,o,s,-9999 $oname2
	ncatted -O -a missing_value,pr,o,s,-9999 $oname2

	ncap2 -v -O -s 'pr=pack(pr,0.1,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-24)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5
	#ncatted -a units,time,o,c,"days since 1990-01-01 00:00:00" $oname5

  # remove temporary files
rm t*_pr_*.nc
done

`rg`

In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}rg_1951.nc" # from step 9
basd_mask="${opath}basd_landmask_pl.nc"
#cdo -f nc4c -z zip expr,tas="(tn >= 0) ? 1 : 0" -seltimestep,1 $basd_file $basd_mask
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}rsds_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=142
n_lons=262

## loop to repeat per variable
for y in {1950..1950}
do
	filename="${ipath}rg_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}rsds_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_rsds_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

	"""
	Added -expr,rg="rg*86400" for consistency between the units in EMO-1 and ERA-5
	
	"""
	cdo -f nc4c -z zip remap,$grid_file,$weight_file -expr,rg="rg*86400" -shifttime,1days $era5_file $era5_file_regrid

	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,0minutes $oname1 $oname2

	ncatted -O -a _FillValue,rg,o,s,-9999 $oname2
	ncatted -O -a missing_value,rg,o,s,-9999 $oname2

	ncap2 -v -O -s 'rg=pack(rg,10000,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-24)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5
	## 1950-1989: ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5

  # remove temporary files (optional, can be used to troubleshoot the issues in the code)
rm t*_rd_*.nc
done

`tn`

In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}tn_1951.nc" # from step 9
basd_mask="${opath}basd_landmask_pl.nc"
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}tasmin_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=142
n_lons=262

## loop to repeat per variable
for y in {1950..1989}
do
	filename="${ipath}tn_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}tasmin_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_tasmin_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

	"""
	Added -expr,tn="tx-273.15" to fix the issue that was made earlier in step 2 which replaced tx with tn and tn with tx.
	
	"""

	cdo -f nc4c -z zip remap,$grid_file,$weight_file -expr,tn="tx-273.15" $era5_file $era5_file_regrid

	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,30minutes $oname1 $oname2

	ncatted -O -a _FillValue,tn,o,s,-9999 $oname2
	ncatted -O -a missing_value,tn,o,s,-9999 $oname2

	ncap2 -v -O -s 'tn=pack(tn,0.1,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-30)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5
	## 1950-1989: ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5

  # remove temporary files
rm t*_tn_*.nc
done

`tx`

In [ ]:
%%bash

ipath="${output_dir}/step_9/"
epath="${output_dir}/step_2/ERA5_for_gapfill/"
opath="${output_dir}/step_10/"

## compute BASD landmask, grid and weights for regridding ERA% (only needed once, the same files are used for other variables)
basd_file="${ipath}tx_1951.nc" # from step 9
basd_mask="${opath}basd_landmask_pl.nc"
grid_file="${opath}basd_grid.txt"
cdo griddes $basd_file > $grid_file
weight_file="${opath}ERA5_weights.nc"
era5_file="${epath}tasmax_ERA5_1951.nc" # from step 2
cdo gennn,$grid_file $era5_file $weight_file

# dimension (adapt n_lats and n_longs to file dimensions divided by 3)
n_times=1
n_lats=142
n_lons=262

## loop to repeat per variable
for y in {1950..1989}
do
	filename="${ipath}tx_${y}.nc" # from step 9
	tile="$(echo $(basename "$filename" .nc) | cut -d'_' -f2)"
	era5_file="${epath}tasmax_ERA5_${tile}.nc" # from step 2
	era5_file_regrid="${epath}regrid_tasmax_ERA5_${tile}.nc"
	oname1="${opath}tb_$(basename "$filename" .nc).nc"
	oname2="${opath}tc_$(basename "$filename" .nc).nc"
	oname3="${opath}td_$(basename "$filename" .nc).nc"
	oname4="${opath}te_$(basename "$filename" .nc).nc"
	oname5="${opath}$(basename "$filename" .nc).nc"

	"""
	Added -expr,tn="tx-273.15" to fix the issue that was made earlier in step 2 which replaced tx with tn and tn with tx.
	
	"""

	cdo -f nc4c -z zip remap,$grid_file,$weight_file -expr,tx="tn-273.15" $era5_file $era5_file_regrid

	cdo -f nc4c -z zip ifthenelse $basd_mask $filename $era5_file_regrid $oname1
	cdo -f nc4c -z zip shifttime,390minutes $oname1 $oname2

	ncatted -O -a _FillValue,tx,o,s,-9999 $oname2
	ncatted -O -a missing_value,tx,o,s,-9999 $oname2

	ncap2 -v -O -s 'tx=pack(tx,0.1,0);' $oname2 $oname3
	ncpdq -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a time,lat,lon $oname3 $oname4
	ncap2 -O -s 'time=(time-24)/24;' $oname4 $oname5
	ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5
	## 1950-1989: ncatted -a units,time,o,c,"days since 1950-01-02 00:00:00" $oname5

  # remove temporary files
rm t*_tx_*.nc
done

## Step 10.3
Calculation of the annual mean. <br/>
You need to SHP file with Province coordinates.

In [ ]:
import geopandas as gpd
import xarray as xr
import rioxarray
import numpy as np
from rasterio.features import geometry_mask
import pandas as pd
import os


input_dir='/LOCATION-TO-YOUR-FOLDER/CLIMB'  # Keep location of Python to compass_framework folder

# Load the shapefile
shapefile_path = '${input_dir}/step_10/shp/voivodeships.shp' 
poland_shapefile = gpd.read_file(shapefile_path)


# List of variables and years you want to process
variables = ['tx', 'tn', 'pd', 'ws', 'rg', 'pr']  # 'tx', 'tn', 'pd', 'ws', 'rg', 'pr'
years = range(1990, 2022)  # range

# Directory where your NetCDF files are located
netcdf_dir = '${input_dir}/step_10/'

# Directory where you want to save the output CSV files
output_dir = '${input_dir}/step_10/mean-y/'

# Make sure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List to store results for CSV output
results = []

# Loop through each variable
for variable in variables:

    # Loop through each year and load the NetCDF file
    for year in years:
        netcdf_file = f'{netcdf_dir}{variable}_{year}.nc'
        
        # Check if file exist
        if not os.path.exists(netcdf_file):
            print(f"File {netcdf_file} not found, skipping this year.")
            continue
        
        # Open the dataset
        data = xr.open_dataset(netcdf_file)
        
        # Access the specific variable (e.g., 'tx' for maximum daily temperature)
        var_data = data[variable]
        
        # Make sure the data has proper geospatial coordinates
        var_data = var_data.rio.write_crs("EPSG:4326")

        # Loop through each voivodeship
        for idx, voivodeship in poland_shapefile.iterrows():
            # Get the geometry (boundary) of the voivodeship
            geometry = [voivodeship['geometry']]
            
            # Create a mask for the voivodeship geometry (lat/lon grid is 2D)
            mask = geometry_mask([geom for geom in geometry], 
                                 transform=var_data.rio.transform(), 
                                 invert=True, 
                                 out_shape=var_data.shape[-2:])
            
            # Apply the mask to the entire time series data
            masked_data = var_data.where(mask)

            # Calculate the mean value at each grid cell across all timestamps
            if variable == 'pr':
                # For 'pr' 
                mean_per_grid_cell = masked_data.mean(dim='time') * 365
            else:
                # For all variables except pr
                mean_per_grid_cell = masked_data.mean(dim='time')

            # Calculate the mean of these values for the entire region within the voivodeship
            mean_max_value = mean_per_grid_cell.mean().item()

            # Add the result to the list
            results.append({
                'variable': variable, 
                'voivodeship': voivodeship['nazwa'],  
                'mean': mean_max_value,  
                'year': year  
            })

# Create a DataFrame from the results
df_results = pd.json_normalize(results)
df_results['mean'] = df_results['mean'].round(4)
df_results = df_results.dropna(subset=['year'])
df_results['year'] = df_results['year'].astype(int).astype(str)

print(df_results.head())
print(df_results.columns)

# Save to CSV, one file per variable
for variable in variables:
    df_var = df_results[df_results['variable'] == variable]
    csv_filename = f'{output_dir}{variable}-mean-y.csv'
    df_var.to_csv(csv_filename, index=False)

print("CSV is ready.")


## Step 11 Lisvap script

To run Step 11 (LISVAP-based PET calculation), you need to use the external LISVAP tool.
LISVAP is not included in this repository because the original source distribution is large (~2 GB).
Instead, please install LISVAP and PCRaster from their official sources and then use the sample configuration files provided in this folder.

⸻

	1.	Install LISVAP and PCRaster

⸻

Follow the official LISVAP installation guide:

https://ec-jrc.github.io/lisflood-lisvap/3_LISVAP_installation/

and the official PCRaster installation guide:

https://pcraster.geo.uu.nl/pcraster/4.4.1/documentation/pcraster_project/install.html

In our workflow, PCRaster is run from a Conda virtual environment (for example named lisvapenv),
and LISVAP is installed via pip inside the same environment. Any equivalent setup is fine,
as long as LISVAP and PCRaster are both available on your PATH.

⸻

	2.	Files provided in this folder (step_11)

⸻

This folder contains:
	•	config.xml
A sample LISVAP configuration file adapted to the CLIMB workflow.
It already contains the correct structure and parameters for reading the output
of Step 10 and producing daily PET fields.
	•	basemap/
A basemap directory prepared for Poland (e.g. required masks, static maps).
LISVAP will use these files for spatial reference and land/sea masking.

You can use these files as a starting point and only adjust the paths to match
your local installation.

⸻

	3.	How to adapt the configuration file

⸻

	- Copy the config.xml from this step_11 folder into your LISVAP working directory (or point LISVAP directly to this file, depending on how you run it).
	- Open config.xml in a text editor and update the path entries so that they match your system:
        •	paths to the input meteorological data (NetCDF files produced by Step 10),
        •	path to the basemap directory (this basemap/ folder for Poland),
        •	path to the output directory where LISVAP should write PET NetCDF files.
	- Save the modified config.xml.

The original example configuration files shipped with LISVAP may not work directly
with the CLIMB data structure. For this reason, it is recommended to start from
the config.xml provided here and only change the paths.

⸻

	4.	Running LISVAP

⸻

After adapting config.xml:
	1.	Activate the environment where LISVAP and PCRaster are installed (e.g. lisvapenv).
	2.	Run LISVAP using the adapted configuration file, following the instructions from the
LISVAP documentation (for example by specifying the config file as input).

Please refer to the official LISVAP manual for the exact command-line invocation,
as it may change between versions.

⸻

	5.	Notes

⸻

•	This repository only provides the configuration and example setup for Step 11; it does not redistribute the LISVAP source code or binaries.
•	The LISVAP step is optional and only needed if you want to derive PET products in a way that is fully consistent with the JRC/LISFLOOD workflow.
•	Make sure that the temporal coverage and spatial grid of your Step 10 outputs are consistent with the configuration used in LISVAP.

Sample config XML is located in SCRIPT folder.<br/> 

Oryginal file from Lisvap project might not work properly. Donwload sample config file from SCRIPT folder and change path to your location. Next replace the downloaded config xml with the one in the script and run LISVAP.

# Possible fixes

This section contains information about possible fixes or suggestions to prevent the issues that were encountered.

### 1.1. Recommended versions on which all the scripts have been tested to run without any issues.

| Software               | Version     |
|:-----------------------|:------------|
| Python                 | 3.11.6      |
| Ubuntu                 | 22.04 ([Check WSL2 documentation](https://learn.microsoft.com/en-us/windows/wsl/install) for Linux on Windows)      |
| Cartopy                | 0.23.0      |
| cdo                    | 2.2.3       |
| cdsapi                 | 0.7.0       |
| certifi                | 2024.7.4    |
| cf-units               | 3.2.0       |
| cftime                 | 1.6.4       |
| comm                   | 0.2.2       |
| contourpy              | 1.2.1       |
| cycler                 | 0.12.1      |
| debugpy                | 1.8.5       |
| decorator              | 5.1.1       |
| executing              | 2.0.1       |
| exsce                  | 1.5.0       |
| ecCodes                | 2.31.1      |
| FILE                   | 1.9.1       |
| fonttools              | 4.53.1      |
| h5netcdf               | 1.2.0       |
| ipykernel              | 6.29.5      |
| ipython                | 8.26.0      |
| ipywidgets             | 8.1.3       |
| jedi                   | 0.19.1      |
| Jinja2                 | 3.1.4       |
| jupyter_client         | 8.6.2       |
| jupyter_core           | 5.7.2       |
| jupyterlab_widgets     | 3.0.11      |
| kiwisolver             | 1.4.5       |
| MarkupSafe             | 2.1.5       |
| matplotlib             | 3.9.1.post1 |
| matplotlib-inline      | 0.1.7       |
| nest-asyncio           | 1.6.0       |
| NetCDF                 | 4.9.2       |
| netCDF4                | 1.7.1.post1 |
| numpy                  | 1.26.4      |
| packaging              | 24.1        |
| pandas                 | 2.2.2       |
| parso                  | 0.8.4       |
| pexpect                | 4.9.0       |
| pillow                 | 10.4.0      |
| pip                    | 24.2        |
| platformdirs           | 4.2.2       |
| prompt_toolkit         | 3.0.47      |
| psutil                 | 6.0.0       |
| ptyprocess             | 0.7.0       |
| pure_eval              | 0.2.3       |
| Pygments               | 2.18.0      |
| pyparsing              | 3.1.2       |
| pyproj                 | 3.6.1       |
| pyshp                  | 2.3.1       |
| python-dateutil        | 2.9.0.post0 |
| pytz                   | 2024.1      |
| pyzmq                  | 26.1.0      |
| scipy                  | 1.14.0      |
| setuptools             | 65.5.0      |
| shapely                | 2.0.5       |
| six                    | 1.16.0      |
| stack-data             | 0.6.3       |
| tabulate               | 0.9.0       |
| tornado                | 6.4.1       |
| traitlets              | 5.14.3      |
| typing_extensions      | 4.12.2      |
| tzdata                 | 2024.1      |
| wcwidth                | 0.2.13      |
| widgetsnbextension     | 4.0.11      |
| xarray                 | 2024.7.0    |
| lisvap                 | [https://ec-jrc.github.io/lisflood-lisvap/3_LISVAP_installation/]    | 
| pcraster               | 4.4.1 [https://pcraster.geo.uu.nl/pcraster/4.4.1/documentation/pcraster_project/install.html]       | 

### Possible error with nco 

In [ ]:
n_times=$(cdo ntime $filename)
ncpdq -4 -O --cnk_plc=g3d --cnk_dmn=time,$n_times --cnk_dmn=lat,$n_lats --cnk_dmn=lon,$n_lons -a lon,lat,time $filename "${output_dir}/step_5/ERA5_$(basename "$filename" .nc).nc"


**Error in terminal:** *ncpdq: ERROR received 9 positional filenames; need exactly two*

**How to fix:** 
Add after ncpdq



In [ ]:
echo "n_times: $n_times"

run script and check results. 
Correct value should display one line, eg.: *n_times: 365*

If you see another line below value:

> *n_times: 365* 

> *cdo ntime: Processed 1 variable [0.01s 44MB].*

add to n_time definition code:

In [ ]:
n_times=$(cdo ntime $filename  | awk 'NR==1 {print $1}')

### 1.2. If you are getting an error in step 7 for `tas` variable not found in the data, follow the below steps:

1. First verify if the variable name is var167 by running the code snippet below in your terminal.

In [ ]:
ncdump -h input_file.nc #replace input file with the tas file that you want to rename

2. Use `ncrename` function from `cdo` library to rename the variable from `var167` to `tas`. 

ncrename -v old_varname,new_varname input_file.nc #replace input file with the tas file that you want to rename

3. Run the script now and it should work. If not, re-check using ncdump -h your_file.nc to verify if variable is correctly renamed to `tas`

### 1.3. Visualizing the data

It is recommended that you visualize the data after completion of each step to verify if the data is being correctly produced. The data can be visualized by use of **ArcGIS** or **QGIS** or by utilizing below Python script.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset

def plot_nc_variable(nc_file, var_name, time_idx=1024):
    dataset = Dataset(nc_file, 'r')
    if var_name not in dataset.variables:
        print(f"Variable '{var_name}' not found in {nc_file}.")
        return
    var_data = dataset.variables[var_name]
    data = var_data[:, :, time_idx].T  
    lats = dataset.variables['lat'][:]
    lons = dataset.variables['lon'][:]

    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.contourf(lon_grid, lat_grid, data, cmap='viridis')
    plt.colorbar(label=f"{var_name}")
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f"{var_name} at time index {time_idx}")
    plt.show()
    dataset.close()

nc_file = '${output_dir}/ERA5_tasrange_ERA5_1990_2022_t.nc'
var_name = 'tasrange'
plot_nc_variable(nc_file, var_name, time_idx=0)

### 1.4. Memory error in Step 9

If you encounter crashes in Python/Jupyter during Step 9 due to insufficient memory, ensure that your system has at least 32 GB of RAM. Should the issue persist, close all active Python/Jupyter instances or restart your computer and attempt running the script again.

To mitigate memory constraints, consider dividing the script into smaller segments and running them individually, except for the temperature and vapor pressure/humidity sections, which must be executed together. However, for optimal performance and to prevent memory-related issues, it is recommended to run the script on a machine or external server with 64 GB or more of RAM.

### 1.5. Troubleshooting in Step 10

To diagnose issues in Step 10, several commands can help identify where the problem arises, particularly related to timestamps, incorrect variables, or inconsistencies in variable units.

1. Retain Temporary Files: Remove the final command in the code that deletes temporary files for specific variables. These files are valuable for troubleshooting. For each variable, four temporary files will be generated with the following naming convention: varname_b, varname_c, varname_d.

2. Use CDO Info: Apply the cdo info command to each temporary file to pinpoint where the issue occurs.

3. Verify Previous Steps: Review the earlier steps to ensure all were completed correctly. If any steps were performed incorrectly, repeat them and then re-execute Step 10.

The most likely issue may be related to timestamps or inconsistent units. These can be resolved manually in Step 10 by adjusting the minutes or ensuring the units are consistent with the EMO-1 data.